<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Pytorch_Straight%20Through%20Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing required libraries
!pip install torchinfo
import torchinfo
import torch
from torch import nn,optim
import torch.nn.functional as F

In [ ]:
# Checking argmax function
torch.argmax(torch.tensor([[32,11,12,14],[1,123,12,212]]), dim = 1)

tensor([0, 3])

In [ ]:
# Creating our own LeNet5

class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) # in channel , out channe, kernel
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d((2,2))

    self.conv2 = nn.Conv2d(6,16,5)  # in channel , out channe, kernel
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d((2,2))

    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    # x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
    # x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = self.conv1(x)
    x = self.relu1(x)
    x= self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x= self.maxpool2(x)

    x = x.view(-1,int(x.nelement() / x.shape[0]))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    x = torch.argmax(x, dim = 1)

    return x



model = LeNet5()



In [ ]:
torchinfo.summary(model, (3, 32, 32), batch_dim = 0,col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 0)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
LeNet5                                   [1, 3, 32, 32]            [1]                       --                        --                        --
├─Conv2d: 1-1                            [1, 3, 32, 32]            [1, 6, 28, 28]            456                       [5, 5]                    357,504
├─ReLU: 1-2                              [1, 6, 28, 28]            [1, 6, 28, 28]            --                        --                        --
├─MaxPool2d: 1-3                         [1, 6, 28, 28]            [1, 6, 14, 14]            --                        [2, 2]                    --
├─Conv2d: 1-4                            [1, 6, 14, 14]            [1, 16, 10, 10]           2,416                     [5, 5]                    241,600
├─ReLU: 1-5                              [1, 16, 10, 10]           [1, 16, 10, 10]           --

In [ ]:
# Loading the dataset
from torchvision import transforms
train_transforms = transforms.Compose([ transforms.RandomCrop(32, padding=4), transforms.ToTensor(), transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])
train_data = CIFAR10(root="./train/", train=True, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader( train_data,batch_size=16, shuffle=True)

Files already downloaded and verified


In [ ]:
# Our loss function
def my_loss(output, target):
    output = torch.tensor(output, dtype=torch.float)
    loss = ((output - target)**2).mean()
    return loss
# Our optimizer
optimizer = optim.SGD(model.parameters(),lr=0.001, momentum=0.9)

In [ ]:
# Training the model

N_EPOCHS = 2
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  for inputs, labels in trainloader:

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = my_loss(outputs, labels.float())
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
  print("Epoch: {} Loss: {}".format(epoch,epoch_loss/len(trainloader)))

<ipython-input-190-36de3bb5f901>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = torch.tensor(output, dtype=torch.float)


RuntimeError: ignored